In [317]:
import numpy as np
import pandas as pd
from typing import List, Dict, Tuple
from dataclasses import dataclass
from datetime import time
import time as tm
import sys

class Stop:
    def __init__(self, name: str, latitude: float, longitude: float) -> None:
        self.name = name
        self.latitude = latitude
        self.longitude = longitude
    def __str__(self) -> str:
        return self.name + "[" + str(self.latitude) + ", " + str(self.longitude) + "]"
    def __repr__(self) -> str:
        return self.name + ", " + str(self.latitude) + "," + str(self.longitude)
    def __hash__(self) -> int:
        return hash(str(self.name))
    def __eq__(self, __value: object) -> bool:
        return isinstance(__value, Stop) and str(self.name) == str(__value.name)
    
@dataclass
class Route:
    line: str
    departure_minutes: int
    arrival_minutes: int

@dataclass
class StopRecord():
    min_arrival_minutes: int
    last_stop: Stop
    last_route: Route
    
def time_to_minutes(time_str: str) -> int:
    time_arr = time_str.split(':')
    hour=int(time_arr[0])
    minute=int(time_arr[1])
    return 60 * (hour % 24) + minute
def format_time(abnormal_time: int) -> str:
    return str(abnormal_time // 60).zfill(2) + ":" + str(abnormal_time % 60).zfill(2)

class Dijkstra():
    def __init__(self, filename="connection_graph (1).csv") -> None:
        self.logs: List[Tuple[str,float]] = [("start", tm.time())]
        self._log("load start")
        self._load(filename)
        self._log("load end")
        self._log("graph creation start")
        self._create()
        self._log("graph creation end")
        
    def _log(self, label: str) -> None:
        self.logs.append((label, tm.time()))
        
    def _print_logs(self) -> None:
        for i in range(len(self.logs)):
            label, timestamp = self.logs[i]
            if i % 2 == 0:
                print(f"--- {label}: {timestamp - self.logs[0][1]} seconds since start \n\t{timestamp - self.logs[max(i-1,0)][1]} seconds since {self.logs[max(i-1,0)][0]}", file=sys.stderr)
        
    def _load(self, filename):
        self.data = pd.read_csv(filename, low_memory=False)
        self.data.set_index(keys='Unnamed: 0', inplace=True)
        self.data.drop_duplicates(ignore_index=True, inplace=True)

    def _create(self):
        self.graph: Dict[Stop, Dict[Stop, List[Route]]] = {}
        for stop in self.data.itertuples():
            start_stop: Stop = Stop(stop.start_stop, stop.start_stop_lat, stop.start_stop_lon) 
            end_stop: Stop = Stop(stop.end_stop, stop.end_stop_lat, stop.end_stop_lon)
            route: Route = Route(line=stop.line, departure_minutes=time_to_minutes(stop.departure_time), arrival_minutes=time_to_minutes(stop.arrival_time))
            if start_stop not in self.graph.keys():
                self.graph[start_stop] = {end_stop: [route]}
            elif end_stop in self.graph[start_stop].keys():
             self.graph[start_stop][end_stop].append(route)
            else:
                self.graph[start_stop][end_stop] = [route] 
            if end_stop not in self.graph.keys():
                self.graph[end_stop] = {}
        for stop, neighbors in self.graph.items():
            for _, routes in neighbors.items():
                routes.sort(key=lambda rt: rt.arrival_minutes)   
    def run(self, a_start: Stop, b_end: Stop, start_time: str, clear_logs: bool= False):
        if clear_logs:
            self.logs = [("start", tm.time())]
        self._log("proceeding start")
        self._proceed(a_start, start_time)
        self._log("proceeding end")
        self._log("printing start")
        self._print(a_start,b_end,start_time)
        self._log("printing end")
        self._print_logs()
    
    def _proceed(self, a_start: Stop, start_time: str):
        change_minutes = 1
        time = time_to_minutes(start_time)
        self.stops_records: Dict[Stop, StopRecord] = {} 
        for key in self.graph.keys():
            self.stops_records[key] = StopRecord(1e10, None, None)
        self.stops_records[a_start] = StopRecord(time, None, None)
        unseen_stops = list(self.graph.keys())
        while(len(unseen_stops) > 0):
            curr_stop = unseen_stops[0]
            for stop in unseen_stops:
                if self.stops_records[stop].min_arrival_minutes < self.stops_records[curr_stop].min_arrival_minutes:
                    curr_stop = stop
            unseen_stops.remove(curr_stop)
            arrival_minutes_modulo = self.stops_records[curr_stop].min_arrival_minutes % (24*60) 
            for neighbor, routes in self.graph[curr_stop].items():
                min_arrival_id = -1
                for i in range(len(routes)):
                    change_fine = 0 if self.stops_records[curr_stop].last_route is None or routes[i].line == self.stops_records[curr_stop].last_route.line else change_minutes
                    if arrival_minutes_modulo + change_fine <= routes[i].departure_minutes:
                        min_arrival_id = i
                        break
                min_arrival_minutes = routes[min_arrival_id].arrival_minutes if min_arrival_id > -1 else 24*60+routes[0].arrival_minutes
                min_arrival_id = max(min_arrival_id,0)
                min_arrival_minutes += (self.stops_records[curr_stop].min_arrival_minutes // (24*60)) * 24*60 
                min_arrival_minutes += 0 if routes[min_arrival_id].arrival_minutes >= routes[min_arrival_id].departure_minutes else 24*60
                if min_arrival_minutes < self.stops_records[neighbor].min_arrival_minutes:
                    self.stops_records[neighbor].min_arrival_minutes = min_arrival_minutes
                    self.stops_records[neighbor].last_stop = curr_stop
                    self.stops_records[neighbor].last_route = routes[min_arrival_id]
                    
    def _print(self, a_start: Stop, b_end: Stop, start_time: str) -> None:
        temp = b_end, self.stops_records[end]
        route: List[Tuple[Stop, StopRecord]] = []
        max_length = [0,0,0,0,0]
        while temp[1].last_stop is not None:
            route.append(temp)
            for i, element in enumerate([temp[1].last_route.line, temp[1].last_stop.name, format_time(temp[1].last_route.departure_minutes), temp[0].name, format_time(temp[1].last_route.arrival_minutes)]):
                max_length[i] = len(str(element)) if len(str(element)) > max_length[i] else max_length[i]
            temp = temp[1].last_stop, self.stops_records[temp[1].last_stop]
        route.reverse()
        for i, stop in enumerate(route):
            day_info = "Day " +  str(stop[1].min_arrival_minutes // (24*60) + 1)
            print(f"{str(i+1).rjust(len(str(len(route))))}. \t{str(stop[1].last_route.line).rjust(max_length[0])}) [{format_time(stop[1].last_route.departure_minutes).rjust(max_length[2])}] {stop[1].last_stop.name.ljust(max_length[1])} - "
                f"[{format_time(stop[1].last_route.arrival_minutes)}] {stop[0].name} ({day_info})")

d = Dijkstra()
def run(a_start: Stop, b_end: Stop, start_time: str) -> None:
    d.run(a_start, b_end, start_time)

In [318]:
d = Dijkstra()

In [319]:
start = Stop("Trawowa",51.09727033,16.94939119)
end = Stop("Stanisławowska (W.K. Formaty)",0,0)
start_time = '23:17:00'
d.run(start, end, start_time)

1. 	107) [23:18] Trawowa - [23:18] Stanisławowska (W.K. Formaty) (Day 1)


--- start: 0.0 seconds since start 
	0.0 seconds since start
--- load end: 1.9308080673217773 seconds since start 
	1.9308080673217773 seconds since load start
--- graph creation end: 4.969882011413574 seconds since start 
	3.039073944091797 seconds since graph creation start
--- proceeding end: 5.247785329818726 seconds since start 
	0.1994791030883789 seconds since proceeding start
--- printing end: 5.24872612953186 seconds since start 
	0.0009407997131347656 seconds since printing start


In [303]:
df = pd.read_csv("connection_graph (1).csv", low_memory=False)

In [315]:
df[(df['start_stop'] == start.name) & (df['end_stop'] == end.name)]

,Unnamed: 0,company,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon
470979,470979,MPK Autobusy,107,05:41:00,05:42:00,Trawowa,Stanisławowska (W.K. Formaty),51.09727,16.949391,51.096946,16.946898
471086,471086,MPK Autobusy,107,08:05:00,08:06:00,Trawowa,Stanisławowska (W.K. Formaty),51.09727,16.949391,51.096946,16.946898
471191,471191,MPK Autobusy,107,10:55:00,10:56:00,Trawowa,Stanisławowska (W.K. Formaty),51.09727,16.949391,51.096946,16.946898
471296,471296,MPK Autobusy,107,13:55:00,13:56:00,Trawowa,Stanisławowska (W.K. Formaty),51.09727,16.949391,51.096946,16.946898
471401,471401,MPK Autobusy,107,16:55:00,16:56:00,Trawowa,Stanisławowska (W.K. Formaty),51.09727,16.949391,51.096946,16.946898
...,...,...,...,...,...,...,...,...,...,...,...
996389,996389,DLA Kąty Wrocławskie,967,07:00:00,07:02:00,Trawowa,Stanisławowska (W.K. Formaty),51.09727,16.949391,51.096946,16.946898
996406,996406,DLA Kąty Wrocławskie,967,13:33:00,13:34:00,Trawowa,Stanisławowska (W.K. Formaty),51.09727,16.949391,51.096946,16.946898
996440,996440,DLA Kąty Wrocławskie,967,15:36:00,15:38:00,Trawowa,Stanisławowska (W.K. Formaty),51.09727,16.949391,51.096946,16.946898
996474,996474,DLA Kąty Wrocławskie,967,07:52:00,07:54:00,Trawowa,Stanisławowska (W.K. Formaty),51.09727,16.949391,51.096946,16.946898


In [316]:
d.graph[start][end]

[Route(line='241', departure_minutes=21, arrival_minutes=21),
 Route(line='241', departure_minutes=81, arrival_minutes=81),
 Route(line='241', departure_minutes=141, arrival_minutes=141),
 Route(line='241', departure_minutes=201, arrival_minutes=201),
 Route(line='241', departure_minutes=261, arrival_minutes=261),
 Route(line='119', departure_minutes=308, arrival_minutes=309),
 Route(line='107', departure_minutes=311, arrival_minutes=312),
 Route(line='241', departure_minutes=321, arrival_minutes=321),
 Route(line='107', departure_minutes=321, arrival_minutes=322),
 Route(line='119', departure_minutes=323, arrival_minutes=324),
 Route(line='119', departure_minutes=328, arrival_minutes=329),
 Route(line='319', departure_minutes=334, arrival_minutes=335),
 Route(line='107', departure_minutes=340, arrival_minutes=341),
 Route(line='107', departure_minutes=341, arrival_minutes=342),
 Route(line='967', departure_minutes=346, arrival_minutes=347),
 Route(line='152', departure_minutes=348, ar

In [269]:
print("Values range for departure time: " + real_timetable['departure_time'].min() + " - " + real_timetable['departure_time'].max())
print("Values range for arrival time: " + real_timetable['arrival_time'].min() + " - " + real_timetable['arrival_time'].max())
print("Values range for start-stop latitude: " + str(real_timetable['start_stop_lat'].min()) + " - " + str(real_timetable['start_stop_lat'].max()))
print("Values range for start-stop longitude: " + str(real_timetable['start_stop_lon'].min()) + " - " + str(real_timetable['start_stop_lon'].max()))

Values range for departure time: 03:34:00 - 30:03:00
Values range for arrival time: 03:36:00 - 30:05:00
Values range for start-stop latitude: 50.98042601 - 51.26556584
Values range for start-stop longitude: 16.695853 - 17.2855453
